# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Jonathan Amme" <br>

GENERAL TODO
- [ ] use consistent scenario order, cf. https://github.com/windnode/WindNODE_ABW/issues/64#issuecomment-670712969

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
####### WINDNODE ########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######### DATA ##########

import re
import pandas as pd

####### Plotting ########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
# papermill parameters will be inserted below this tagged cell

TODO
- [ ] suppress long log output of analysis

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=scenarios,
                                      force_new_results=force_new_results)

In [ ]:
##### Usefull stuff ######

# Names of Municialities
MUN_NAMES = regions_scns[list(scenarios)[-2]].muns.gen.to_dict()
# extend for total ABW region
MUN_NAMES.update({100:'ABW'})
# Colormap
CMAP = px.colors.sequential.GnBu_r
#CMAP_NAME_KEY_PLOT = 'GnBu_r'
CMAP_NAME_KEY_PLOT = 'viridis'

# 0 Key Results

The following plots provide an overview on the key results of all scenarios.

TODO
- [ ] add explanations on params
- [ ] rework autarky as soon as #135 is merged
- [ ] fix title position

In [ ]:
plot_key_scenario_results(results_scns, scenarios, CMAP_NAME_KEY_PLOT)

# 1 Demand and Generation

## 1.1 Installed Capacities Electricity/Heat

TODO
- [ ] Fix title overlap
- [ ] Scenario order?
- [ ] Fix Autarky as soon as PR 135 is done

In [ ]:
cap_el = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity electricity supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_el = cap_el.assign(sum=cap_el.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1) \
    .rename(columns=PRINT_NAMES)

fig = go.Figure()

for tech, data in cap_el.iteritems():
    fig.add_trace(go.Bar(x=cap_el.index,
                         y=data,
                         name=tech,
                         #orientation='h',
                         showlegend=True))

fig.update_layout(
    title='Installed Capacity Electricity Supply',
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)

fig.show()

In [ ]:
cap_th = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity heat supply'].sum(axis=0)
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

cap_th = cap_th.assign(sum=cap_th.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1) \
    .rename(columns=PRINT_NAMES)

fig = go.Figure()

for tech, data in cap_th.iteritems():
    fig.add_trace(go.Bar(x=cap_th.index,
                         y=data,
                         name=tech,
                         #orientation='h',
                         showlegend=True))

fig.update_layout(
    title='Installed Capacity Heat Supply',
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 1.2 Electricity and Heat Generation

In [ ]:
gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
gen_th = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen = pd.concat([gen_el, gen_th], axis=0, keys=['Electricity','Heat'])
df_gen = df_gen / 1e3
df_gen = df_gen.rename(columns=PRINT_NAMES)
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01, subplot_titles=['Electricity','Heat'])
for sector, df in df_gen.groupby(level=0):
    row = 1 if sector == 'Electricity' else 2

    for tech, data in df.loc[sector].iteritems():
        fig.add_trace(go.Bar(y=data.index,
                             x=data,
                             name=tech,
                             orientation='h',
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=True),row=row, col=1)
fig.update_layout(
    title='Generation',
    barmode='stack',
    hovermode="y unified",
    height=100*len(scenarios),
    autosize=True)

fig.update_xaxes(title_text="GWH", row=2, col=1)
fig.show()

In [ ]:
gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
gen_th = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen = pd.concat([gen_el, gen_th], axis=0, keys=['Electricity','Heat'])
df_gen = df_gen.rename(columns=PRINT_NAMES)
df_gen = df_gen / 1e3

fig = make_subplots(rows=len(scenarios), cols=1, shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_gen.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(y=list(zip(*data.index.swaplevel())),
                             x=data,
                             name=tech,
                             orientation='h',
                             #marker_color=CMAP[i+1],
                             legendgroup=tech,
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=not bool(row)), row=row+1, col=1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    height=150*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_xaxes(title_text="GWH", row=len(scenarios), col=1, matches='x')
fig.show() 

In [ ]:
gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
gen_th = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen = pd.concat([gen_el, gen_th], axis=0, keys=['Electricity','Heat'])
df_gen = df_gen.rename(columns=PRINT_NAMES)
df_gen = df_gen / 1e3

fig = make_subplots(rows=1, cols=len(scenarios), shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_gen.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(x=data.index,#.swaplevel(),
                             y=data,
                             name=tech,
                             orientation='v',
                             #marker_color=CMAP[i+1],
                             legendgroup=tech,
                             hovertemplate='%{y:.1f} GWh',
                             showlegend=not bool(row)), row=1, col=row+1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    #height=150*len(scenarios),
    width=150*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_yaxes(title_text="GWH", col=1, row=1, matches='x')
fig.update_xaxes(type='category', tickangle=45)
fig.show() 

## 1.3 Electricity and Heat Demand

In [ ]:
df_el = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Stromnachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Wärmenachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = df_heat.T.groupby([s.split('_')[0] for s in df_heat.columns.values]).sum().T

df_demand = pd.concat([df_el, df_heat], axis=0, keys=['Electricity', 'Heat'])
df_demand = df_demand.rename(columns=PRINT_NAMES)
df_demand = df_demand / 1e3

fig = make_subplots(rows=len(scenarios), cols=1, shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_demand.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(y=list(zip(*data.index.swaplevel())),
                             x=data,
                             name=tech,
                             orientation='h',
                             marker_color=CMAP[i+1],
                             legendgroup=tech,
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=not bool(row)), row=row+1, col=1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    height=150*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_xaxes(title_text="GWH", row=len(scenarios), col=1, matches='x')
fig.show() 

# 2 Area required by RES

### false plots / templates

In [ ]:
df_area = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Area required'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

fig = go.Figure()
for i, (tech, data) in enumerate(df_area.iteritems()):
    fig.add_trace(go.Bar(y=df_area.index,
                         x=data,
                         name=tech,
                         orientation='h',
                         marker_color=CMAP[i+1],
                         hovertemplate='%{x:.1f} ha',
                         showlegend=True))

fig.update_layout(
    title='Required Area',
    barmode='group',
    hovermode="y unified",
    height=200*len(scenarios),
    autosize=True)
fig.update_xaxes(title_text="ha")
fig.show()

In [ ]:
df_data = results_scns[list(scenarios)[0]]['highlevel_results']
traces = ['PV rooftop', 'PV ground', 'rel. wind']

fig = go.Figure()

for i, trace in enumerate(traces):
    mask = [i for i in df_data.index if trace in  i[0]]
    data = df_data.loc[mask]
    index = data.index.get_level_values(level=0)

    fig.add_trace(
        go.Bar(y=index, x=data.values,
               orientation='h',
               name=trace,
               marker_color=CMAP[2*i+1],
              hovertemplate='%{x:.1f} %'))
    
fig.update_layout(title_text = 'Relative Required Area',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)
fig.update_xaxes(showspikes=True)

fig.show()

## 2.1 Available areas: Scenarios

### 2.1.1 Available areas: Wind Energy

### 2.1.2 Available areas: Ground-Mounted PV

### 2.1.3 Available areas: Roof-Mounted PV

## 2.2 Area usage